In [1]:
import os,zipfile

files=zipfile.ZipFile('models.zip','r')
files.extractall('/home/ma-user/work')
files.close()

In [2]:
!pip install dlib

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 3.2 MB 57.0 MB/s eta 0:00:01                        | 747 kB 57.0 MB/s eta 0:00:01  | 2.8 MB 57.0 MB/s eta 0:00:01
  Created wheel for dlib: filename=dlib-19.21.0-cp36-cp36m-linux_x86_64.whl size=3994852 sha256=8f024335a9ff1efe8c7bfb3bf5cf218430c80d32c6ed01e31af3c6acb58d0703
  Stored in directory: /home/ma-user/.cache/pip/wheels/13/3e/12/94e41b59715651d0adbf169499dc3fd294a93b25ec360592cb
Successfully built dlib
You should consider upgrading via the '/home/ma-user/anaconda3/envs/TensorFlow-2.1.0/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip3 install opencv-contrib-python -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 55.6 MB 224 kB/s eta 0:00:012K     |▍                               | 768 kB 39.8 MB/s eta 0:00:02 |█████▊                          | 9.9 MB 483 kB/s eta 0:01:35kB/s eta 0:01:15/s eta 0:00:56  | 30.3 MB 14.9 MB/s eta 0:00:02MB/s eta 0:00:02MB/s eta 0:00:10███████████████▌          | 37.4 MB 2.2 MB/s eta 0:00:09█████████████▍        | 40.7 MB 2.2 MB/s eta 0:00:070:22��███▉    | 48.4 MB 471 kB/s eta 0:00:16��█████████████████████████▋  | 51.5 MB 471 kB/s eta 0:00:09�██▉| 55.3 MB 478 kB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/TensorFlow-2.1.0/bin/python -m pip install --upgrade pip' command.


安装完后检查OpenCV的版本

In [3]:
import cv2
cv2.__version__

'4.4.0'

In [4]:
from __future__ import division
import cv2
import time
import sys
import dlib
import os
import numpy as np

In [5]:
def detectFaceDlibMMOD(detector, frame, inHeight=300, inWidth=0):

    frameDlibMMOD = frame.copy()
    #提取图片尺寸
    frameHeight = frameDlibMMOD.shape[0]
    frameWidth = frameDlibMMOD.shape[1]
    if not inWidth:
        inWidth = int((frameWidth / frameHeight)*inHeight)

    #确定图片的缩放比例
    scaleHeight = frameHeight / inHeight
    scaleWidth = frameWidth / inWidth

    #对图片进行缩放
    frameDlibMMODSmall = cv2.resize(frameDlibMMOD, (inWidth, inHeight))

    frameDlibMMODSmall = cv2.cvtColor(frameDlibMMODSmall, cv2.COLOR_BGR2RGB)
    
    #进行人脸识别
    faceRects = detector(frameDlibMMODSmall, 0)

    #print(frameWidth, frameHeight, inWidth, inHeight)
    bboxes = []
    for faceRect in faceRects:
        #提取人脸的边界坐标
        cvRect = [int(faceRect.rect.left()*scaleWidth), int(faceRect.rect.top()*scaleHeight),
                  int(faceRect.rect.right()*scaleWidth), int(faceRect.rect.bottom()*scaleHeight) ]
        bboxes.append(cvRect)
        #将识别结果绘制在图片上
        cv2.rectangle(frameDlibMMOD, (cvRect[0], cvRect[1]), (cvRect[2], cvRect[3]), (0, 255, 0), int(round(frameHeight/150)), 4)
    return frameDlibMMOD, bboxes

In [6]:
def detectFaceDlibHog(detector, frame, inHeight=300, inWidth=0):

    frameDlibHog = frame.copy()
    frameHeight = frameDlibHog.shape[0]
    frameWidth = frameDlibHog.shape[1]
    if not inWidth:
        inWidth = int((frameWidth / frameHeight)*inHeight)

    scaleHeight = frameHeight / inHeight
    scaleWidth = frameWidth / inWidth
    # scaleHeight=1
    # scaleWidth=1
    

    frameDlibHogSmall = cv2.resize(frameDlibHog, (inWidth, inHeight))

    frameDlibHogSmall = cv2.cvtColor(frameDlibHogSmall, cv2.COLOR_BGR2RGB)
    # frameDlibHogSmall = cv2.cvtColor(frameDlibHog, cv2.COLOR_BGR2RGB)
    faceRects = detector(frameDlibHogSmall, 0)
    #print(frameWidth, frameHeight, inWidth, inHeight)
    bboxes = []
    for faceRect in faceRects:

        cvRect = [int(faceRect.left()*scaleWidth), int(faceRect.top()*scaleHeight),
                  int(faceRect.right()*scaleWidth), int(faceRect.bottom()*scaleHeight) ]
        bboxes.append(cvRect)
        cv2.rectangle(frameDlibHog, (cvRect[0], cvRect[1]), (cvRect[2], cvRect[3]), (0, 255, 0), int(round(frameHeight/150)), 4)
    return frameDlibHog, bboxes

In [7]:
def detectFaceOpenCVDnn(net, frame):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    #去掉改尺寸的部分
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], False, False)
    # blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0,frameOpencvDnn.shape, [104, 117, 123], False, False)
    
    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes

In [8]:
def detectFaceOpenCVHaar(faceCascade, frame, inHeight=300, inWidth=0):
    frameOpenCVHaar = frame.copy()
    frameHeight = frameOpenCVHaar.shape[0]
    frameWidth = frameOpenCVHaar.shape[1]
    if not inWidth:
        inWidth = int((frameWidth / frameHeight) * inHeight)

    scaleHeight = frameHeight / inHeight
    scaleWidth = frameWidth / inWidth
    # scaleHeight=1
    # scaleWidth=1

    frameOpenCVHaarSmall = cv2.resize(frameOpenCVHaar, (inWidth, inHeight))
    frameGray = cv2.cvtColor(frameOpenCVHaarSmall, cv2.COLOR_BGR2GRAY)
    # frameGray = cv2.cvtColor(frameOpenCVHaar, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(frameGray)
    bboxes = []
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x + w
        y2 = y + h
        cvRect = [int(x1 * scaleWidth), int(y1 * scaleHeight),
                  int(x2 * scaleWidth), int(y2 * scaleHeight)]
        bboxes.append(cvRect)
        cv2.rectangle(frameOpenCVHaar, (cvRect[0], cvRect[1]), (cvRect[2], cvRect[3]), (0, 255, 0),
                      int(round(frameHeight / 150)), 4)
    return frameOpenCVHaar, bboxes

In [9]:
def facedetect_compare(image_name):
    
    frame=cv2.imread(image_name)
    frame_count = 0
    tt_opencvHaar = 0
    tt_opencvDnn = 0
    tt_dlibHog = 0
    tt_dlibMmod = 0   
    
    t = time.time()
    outOpencvHaar, bboxes = detectFaceOpenCVHaar(faceCascade, frame)
    tt_opencvHaar += time.time() - t
    fpsOpencvHaar = frame_count / tt_opencvHaar

    label = "OpenCV Haar ; Time : {:.2f}".format(tt_opencvHaar)
    cv2.putText(outOpencvHaar, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

    t = time.time()
    outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,frame)
    tt_opencvDnn += time.time() - t
    fpsOpencvDnn = frame_count / tt_opencvDnn
    label = "OpenCV DNN ; Time : {:.2f}".format(tt_opencvDnn)
    cv2.putText(outOpencvDnn, label, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

    t = time.time()
    outDlibHog, bboxes = detectFaceDlibHog(hogFaceDetector,frame)
    tt_dlibHog += time.time() - t
    fpsDlibHog = frame_count / tt_dlibHog

    label = "DLIB HoG ; ; Time : {:.2f}".format(tt_dlibHog)
    cv2.putText(outDlibHog, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

    t = time.time()
    outDlibMMOD, bboxes = detectFaceDlibMMOD(dnnFaceDetector,frame)
    tt_dlibMmod += time.time() - t
    fpsDlibMmod = frame_count / tt_dlibMmod

    label = "DLIB MMOD ; Time : {:.2f}".format(tt_dlibMmod)
    cv2.putText(outDlibMMOD, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

    top = np.hstack([outOpencvHaar, outOpencvDnn])
    bottom = np.hstack([outDlibHog, outDlibMMOD])
    combined = np.vstack([top, bottom])
    cv2.imwrite('Compare-'+image_name, combined)

In [10]:
#Dlib MMOD视频处理
def videoface_DlibMMOD(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()       
    vid_writer = cv2.VideoWriter('Dlib MMOD-output-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))
    frame_count = 0
    tt_dlibMmod = 0 
    while cap.isOpened():   
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break        
        t = time.time()
        outDlibMMOD, bboxes = detectFaceDlibMMOD(dnnFaceDetector,frame)
        tt_dlibMmod += time.time() - t
        fpsDlibMmod = frame_count / tt_dlibMmod

        label = "DLIB MMOD ; Time : {:.2f}".format(tt_dlibMmod)
        cv2.putText(outDlibMMOD, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

        
        if frame_count == 1:
            
            tt_dlibMmod = 0
        vid_writer.write(outDlibMMOD)
        k = cv2.waitKey(10)

In [11]:
#Dlib hog视频处理
def videoface_DlibHog(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('DlibHog-output-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))
    frame_count = 0
    tt_dlibHog = 0
    
    while cap.isOpened():         
    
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        t = time.time()
        outDlibHog, bboxes = detectFaceDlibHog(hogFaceDetector,frame)
        tt_dlibHog += time.time() - t
        fpsDlibHog = frame_count / tt_dlibHog

        label = "DLIB HoG ; ; Time : {:.2f}".format(tt_dlibHog)
        cv2.putText(outDlibHog, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

        
        if frame_count == 1:
           
            tt_dlibHog = 0
            
        vid_writer.write(outDlibHog)
        k = cv2.waitKey(10)

In [12]:
#OpenCV Haar视频处理
def videoface_OpenCVHaar(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('OpenCVHaar-output-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))
    frame_count = 0
    tt_opencvHaar = 0
    
    while cap.isOpened():         
    
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        t = time.time()
        outOpencvHaar, bboxes = detectFaceOpenCVHaar(faceCascade, frame)
        tt_opencvHaar += time.time() - t
        fpsOpencvHaar = frame_count / tt_opencvHaar

        label = "OpenCV Haar ; Time : {:.2f}".format(tt_opencvHaar)
        cv2.putText(outOpencvHaar, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

        
        if frame_count == 1:
            tt_opencvHaar = 0
            
        vid_writer.write(outOpencvHaar)
        k = cv2.waitKey(10)
    

In [13]:
#OpenCV Dnn视频处理
def videoface_OpenCVDnn(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('OpenCVDnn-output-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))
    frame_count = 0
    tt_opencvDnn = 0
    
    while cap.isOpened():         
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        
        t = time.time()
        outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,frame)
        tt_opencvDnn += time.time() - t
        fpsOpencvDnn = frame_count / tt_opencvDnn
        label = "OpenCV DNN ; Time : {:.2f}".format(tt_opencvDnn)
        cv2.putText(outOpencvDnn, label, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, cv2.LINE_AA)

        
    #cv2.imwrite('Compare-'+image_name, combined)
        if frame_count == 1:
            
            tt_opencvDnn = 0
            
        vid_writer.write(outOpencvDnn)
        k = cv2.waitKey(10)

In [14]:
def opencv_compare(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('opencv-compar-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1]*2,frame.shape[0]))
    frame_count = 0
    tt_opencvHaar = 0
    tt_opencvDnn = 0
    tt_dlibHog = 0
    tt_dlibMmod = 0 
    while cap.isOpened(): 
        
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        t = time.time()
        outOpencvHaar, bboxes = detectFaceOpenCVHaar(faceCascade, frame)
        tt_opencvHaar += time.time() - t
        fpsOpencvHaar = frame_count / tt_opencvHaar

        label = "OpenCV Haar ; FPS : {:.2f}".format(tt_opencvHaar)
        cv2.putText(outOpencvHaar, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        t = time.time()
        outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,frame)
        tt_opencvDnn += time.time() - t
        fpsOpencvDnn = frame_count / tt_opencvDnn
        label = "OpenCV DNN ; FPS : {:.2f}".format(tt_opencvDnn)
        cv2.putText(outOpencvDnn, label, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)



        top = np.hstack([outOpencvHaar, outOpencvDnn])
        
        
    #cv2.imwrite('Compare-'+image_name, combined)
        if frame_count == 1:
            tt_opencvHaar = 0
            tt_opencvDnn = 0
            tt_dlibHog = 0
            tt_dlibMmod = 0
        vid_writer.write(top)
    vid_writer.release()

In [15]:
def dlib_compare(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('dlib_compare-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1]*2,frame.shape[0]*2))
    frame_count = 0
    tt_opencvHaar = 0
    tt_opencvDnn = 0
    tt_dlibHog = 0
    tt_dlibMmod = 0 
    while cap.isOpened(): 
        
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        

        t = time.time()
        outDlibHog, bboxes = detectFaceDlibHog(hogFaceDetector,frame)
        tt_dlibHog += time.time() - t
        fpsDlibHog = frame_count / tt_dlibHog

        label = "DLIB HoG ; ; FPS : {:.2f}".format(tt_dlibHog)
        cv2.putText(outDlibHog, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        t = time.time()
        outDlibMMOD, bboxes = detectFaceDlibMMOD(dnnFaceDetector,frame)
        tt_dlibMmod += time.time() - t
        fpsDlibMmod = frame_count / tt_dlibMmod

        label = "DLIB MMOD ; FPS : {:.2f}".format(tt_dlibMmod)
        cv2.putText(outDlibMMOD, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        #top = np.hstack([outOpencvHaar, outOpencvDnn])
        bottom = np.hstack([outDlibHog, outDlibMMOD])
        #combined = np.vstack([top, bottom])
    #cv2.imwrite('Compare-'+image_name, combined)
        if frame_count == 1:
            tt_opencvHaar = 0
            tt_opencvDnn = 0
            tt_dlibHog = 0
            tt_dlibMmod = 0
        vid_writer.write(bottom)

In [16]:
def videoface_compare(vedio_name):
    cap = cv2.VideoCapture(vedio_name)
    hasFrame, frame = cap.read()
       
    vid_writer = cv2.VideoWriter('output-{}.avi'.format(str(vedio_name).split(".")[0]),cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1]*2,frame.shape[0]*2))
    frame_count = 0
    tt_opencvHaar = 0
    tt_opencvDnn = 0
    tt_dlibHog = 0
    tt_dlibMmod = 0 
    while cap.isOpened(): 
        
        hasFrame, frame = cap.read()
        if hasFrame!= True:
             break    
        t = time.time()
        outOpencvHaar, bboxes = detectFaceOpenCVHaar(faceCascade, frame)
        tt_opencvHaar += time.time() - t
        fpsOpencvHaar = frame_count / tt_opencvHaar

        label = "OpenCV Haar ; FPS : {:.2f}".format(tt_opencvHaar)
        cv2.putText(outOpencvHaar, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        t = time.time()
        outOpencvDnn, bboxes = detectFaceOpenCVDnn(net,frame)
        tt_opencvDnn += time.time() - t
        fpsOpencvDnn = frame_count / tt_opencvDnn
        label = "OpenCV DNN ; FPS : {:.2f}".format(tt_opencvDnn)
        cv2.putText(outOpencvDnn, label, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        t = time.time()
        outDlibHog, bboxes = detectFaceDlibHog(hogFaceDetector,frame)
        tt_dlibHog += time.time() - t
        fpsDlibHog = frame_count / tt_dlibHog

        label = "DLIB HoG ; ; FPS : {:.2f}".format(tt_dlibHog)
        cv2.putText(outDlibHog, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        t = time.time()
        outDlibMMOD, bboxes = detectFaceDlibMMOD(dnnFaceDetector,frame)
        tt_dlibMmod += time.time() - t
        fpsDlibMmod = frame_count / tt_dlibMmod

        label = "DLIB MMOD ; FPS : {:.2f}".format(tt_dlibMmod)
        cv2.putText(outDlibMMOD, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, cv2.LINE_AA)

        top = np.hstack([outOpencvHaar, outOpencvDnn])
        bottom = np.hstack([outDlibHog, outDlibMMOD])
        combined = np.vstack([top, bottom])
    #cv2.imwrite('Compare-'+image_name, combined)
        if frame_count == 1:
            tt_opencvHaar = 0
            tt_opencvDnn = 0
            tt_dlibHog = 0
            tt_dlibMmod = 0
        vid_writer.write(combined)

In [17]:
#m模型的初始化
#OpenCV Haar
faceCascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')

#OpenCV Dnn模型加载
modelFile = "./models/opencv_face_detector_uint8.pb"
configFile = "./models/opencv_face_detector.pbtxt"
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

conf_threshold = 0.7

# DLIB HoG
hogFaceDetector = dlib.get_frontal_face_detector()

# DLIB MMOD
dnnFaceDetector = dlib.cnn_face_detection_model_v1("./models/mmod_human_face_detector.dat")

In [21]:
opencv_compare('Dengchao.mp4')

In [21]:
dlib_compare('Dengchao.mp4')

In [20]:
videoface_compare('Dengchao.mp4')

In [23]:
videoface_OpenCVHaar('Dengchao.mp4')

In [25]:
videoface_DlibHog('Dengchao.mp4')